In [ ]:
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.nddata import CCDData
from astropy import units as u
from astropy.timeseries import BinnedTimeSeries

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [3]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def parse_timeseries_data(images_dir):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    timestamps = []
    fluxes = []
    for i, path in np.ndenumerate(images):
        image = CCDData.read(path, unit='adu')
        timestamps.append(image.header['DATE-OBS'])
        fluxes.append(image.header['FLUX'])
    
    light_curve = BinnedTimeSeries(
        time_bin_start=timestamps,
        time_bin_size=15*u.s,
        data={
            'DAO flux' : fluxes * u.adu
        }
    )
        
    return light_curve

In [4]:
matplotlib.use('pgf')
matplotlib.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['font.size'] = 12

In [5]:
cc_lightcurve = BinnedTimeSeries.read(
    './results/cccom_results.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
plt.plot(cc_lightcurve.time_bin_start.jd, cc_lightcurve['DAO flux'] / cc_lightcurve['DAO flux'].max(), 'k.', markersize=4, label='DAO rel. flux')
plt.plot(cc_lightcurve.time_bin_start.jd, cc_lightcurve['AP flux'] / cc_lightcurve['AP flux'].max(), 'r.', markersize=4, label='AP rel. flux')
plt.legend()
plt.xlabel('Julian Date')
plt.ylabel('Relative Flux, Normalized')
plt.title('V* CC Com Light Curve')
plt.savefig('../paper/figures/cc-com_lightcurve.png')

In [6]:
tz_lightcurve = BinnedTimeSeries.read(
    './results/tzboo_results.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
plt.plot(tz_lightcurve.time_bin_start.jd, tz_lightcurve['DAO flux'] / tz_lightcurve['DAO flux'].max(), 'k.', markersize=4, label='DAO rel. flux')
plt.plot(tz_lightcurve.time_bin_start.jd, tz_lightcurve['AP flux'] / tz_lightcurve['AP flux'].max(), 'r.', markersize=4, label='AP rel. flux')
plt.legend()
plt.xlabel('Julian Date')
plt.ylabel('Relative Flux, Normalized')
plt.title('TZ Boo Light Curve')
plt.savefig('../paper/figures/tz-boo_lightcurve.png')